# T matrix for electromagnetic scattering

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
%matplotlib inline
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [2]:
d0 = [0,0,-1] # wave's travel direction
p0 = [1,0,0] # polarization 
k = 5 # wavenumber
deg = 10

In [3]:
def coeff_inc(p,q):
    alpha_q = (0.5*(-1j)**q)*(2*q+1)/(q*(q+1))
    c_q_neg1 = np.sqrt(((2*q+1)/(4*np.pi))*(math.factorial(q-1)/math.factorial(q+1)))
    c_q_pos1 = -c_q_neg1
    if p == -1:
        return (-1j)*alpha_q / c_q_neg1,  alpha_q / c_q_neg1
    elif p == 1:
        return (1j)*alpha_q / c_q_pos1,  alpha_q / c_q_pos1
    else:
        return 0, 0

In [3]:
def sph_b(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)  

def sph_b_dr(q,x):
    r = np.linalg.norm(x)
    return k*(sph_b(q-1,x) - ((q+1)/(k*r))*sph_b(q,x))

def sph_b_dr2(q,x):
    r = np.linalg.norm(x)
    return (k**2)*(0.25 * sph_b(q-2,x) + (3 - 2*(k*r)**2)*sph_b(q,x) / (4*((k*r)**2)) + (1/(2*k*r))*sph_b(q+1,x) + 0.25 * sph_b(q+2,x) - (1/(2*k*r))*sph_b(q-1,x))

In [5]:
def sph(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)
    
def sph_dazi(p, q, x):
    return 1j * p * sph(p, q, x)

def sph_dazi2(p, q, x):
    return (-1) * p**2 * sph(p, q, x)

In [7]:
def sph_dpolar(p, q, x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = sph(p, q, x)
    
    if abs(p) == q and p >= 0:
        return (p / np.tan(polar)) * y1 
    elif abs(p) != q and p >= 0:
        y2 = sph(p + 1, q, x)
        return (p / np.tan(polar)) * y1 - np.sqrt((q - p) * (q + p + 1)) * np.exp(-1j * azimuth) * y2
    elif abs(p) == q and p < 0:
        return (-p / np.tan(polar)) * y1 
    elif abs(p) != q and p < 0:
        y4 = sph(p - 1, q, x)
        return ((-p / np.tan(polar)) * y1 + np.sqrt((q - (-p)) * (q + (-p) + 1)) * np.exp(1j * azimuth) * y4)
    
def sph_dpolar_dazi(p,q,x):
    '''First azi, then polar'''
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = sph(p, q, x)
    
    if abs(p) == q and p >= 0:
        return (1j*p)*((p / np.tan(polar)) * y1 )
    elif abs(p) != q and p >= 0:
        y2 = sph(p + 1, q, x)
        return (1j*p)*((p / np.tan(polar)) * y1 - np.sqrt((q - p) * (q + p + 1)) * np.exp(-1j * azimuth) * y2)
    elif abs(p) == q and p < 0:
        return (1j*p)*((-p / np.tan(polar)) * y1 )
    elif abs(p) != q and p < 0:
        y4 = sph(p - 1, q, x)
        return (1j*p)*(((-p / np.tan(polar)) * y1 + np.sqrt((q - (-p)) * (q + (-p) + 1)) * np.exp(1j * azimuth) * y4))
    
def sph_dazi_dpolar(p,q,x):
    '''First polar, then azi'''
    return sph_dpolar_dazi(p, q, x)

def sph_dpolar2(p,q,x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = sph(p, q, x)
    
    if abs(p) == q and p >= 0:
        return p*(p/(np.tan(polar))**2 - (1/np.sin(polar))**2) * y1
    elif abs(p) == q-1 and p >= 0:
        y2 = sph(p+1, q, x)
        return p*(p/(np.tan(polar))**2 - (1/np.sin(polar))**2) * y1 - np.sqrt((q - p)*(q + p + 1))*(2*p+1)* np.exp(-1j*azimuth) * y2 / np.tan(polar)
    elif abs(p) != q and abs(p) != q-1 and p >= 0:
        y2 = sph(p+1, q, x)
        y3 = sph(p+2, q, x)
        return p*(p/(np.tan(polar))**2 - (1/np.sin(polar))**2) * y1 - np.sqrt((q - p)*(q + p + 1))*(2*p+1)* np.exp(-1j*azimuth) * y2 / np.tan(polar) + \
               np.sqrt((q - p)*(q - p - 1)*(q + p + 2)*(q + p + 1)) * np.exp(-2*1j*azimuth)*y3

    elif abs(p) == q and p < 0:
        return (p*(p/(np.tan(polar))**2 + (1/np.sin(polar))**2)) * y1
    elif abs(p) == q-1 and p < 0:
        y5 = sph(p-1,q,x)
        return (p*(p/(np.tan(polar))**2 + (1/np.sin(polar))**2)) * y1 + np.sqrt((q - (-p))*(q + (-p) + 1))*(2*(-p)+1)* np.exp(1j*azimuth) * y5 / np.tan(polar)
    elif abs(p) != q and abs(p) != q-1 and p < 0:
        y5 = sph(p-1,q,x)
        y6 = sph(p-2,q,x)
        return  (p*(p/(np.tan(polar))**2 + (1/np.sin(polar))**2)) * y1 + np.sqrt((q - (-p))*(q + (-p) + 1))*(2*(-p)+1)* np.exp(1j*azimuth) * y5 / np.tan(polar) \
                + np.sqrt((q - (-p))*(q - (-p) - 1)*(q + (-p) + 2)*(q + (-p) + 1)) * np.exp(2*1j*azimuth)*y6

In [9]:
def sph_wf_M(p,q,x):
    r = np.linalg.norm(x)
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/r)
    
    r_ = -sph_b(q,x)*((np.cos(azimuth) - np.cos(2*polar)/np.sin(polar))*sph(p,q,x) + np.sin(azimuth)*sph_dazi(p,q,x) - np.cos(polar)*sph_dpolar(p,q,x)) 
    polar_ = -sph(p,q,x)*(sph_b(q,x)*(2*np.cos(polar) + np.sin(azimuth)) + np.cos(polar)*r*sph_b_dr(q,x)) + sph_b(q,x)*np.cos(azimuth)*sph_dazi(p,q,x)
    azimuth_ = sph(p,q,x)*(sph_b(q,x)*(2*np.sin(polar)*np.sin(azimuth) - np.cos(polar)*np.cos(azimuth)) + r*sph_b_dr(q,x)*np.sin(polar)*np.sin(azimuth)) \
                - sph_b(q,x)*np.cos(azimuth)*np.sin(polar)*sph_dpolar(p,q,x)
    return [r_*np.sin(polar_)*np.cos(azimuth_), r_*np.sin(polar_)*np.sin(azimuth_), r_*np.cos(polar)]

In [10]:
def sph_wf_N(p,q,x):
    r = np.linalg.norm(x)
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/r)
    
    r_ = (1/r)*(2*sph(p,q,x)*(sph_b(q,x)*(np.cos(azimuth)*np.sin(polar) + 2*np.cos(polar)*np.sin(azimuth)) + r*np.cos(polar)*np.sin(azimuth)*sph_b_dr(q,x)) \
         + r*sph_b_dr(q,x)*((1/np.tan(polar))*sph_dazi(p,q,x) + np.sin(polar)*np.sin(azimuth)*sph_dpolar(p,q,x)) \
         + sph_b(q,x)*(2*(1/np.sin(polar))*(np.cos(polar) + np.sin(azimuth))*sph_dazi(p,q,x) \
            - np.sin(polar)*(np.cos(azimuth)*(1/(np.sin(polar))**2)*sph_dazi2(p,q,x) + (3*np.cos(azimuth)*(1/np.tan(polar)) - 2*np.sin(azimuth))*sph_dpolar(p,q,x)\
            + np.cos(azimuth)*sph_dpolar2(p,q,x))))
    
    polar_ = (1/r)*(sph(p,q,x)*np.sin(polar)*(sph_b(q,x)*(np.cos(azimuth)*(1/np.tan(polar)) + (-1 + (1/np.tan(polar))**2)*np.sin(azimuth)) \
               + r*((np.cos(azimuth)*(1/np.tan(polar)) - 4*np.sin(azimuth))*sph_b_dr(q,x) - r*np.sin(azimuth)*sph_b_dr2(q,x))) \
                    + r*np.cos(azimuth)*np.sin(polar)*sph_b_dr(q,x)*sph_dpolar(p,q,x) \
             + sph_b(q,x)*((-1 + (1/np.tan(polar))**2 - 2*np.cos(azimuth)/np.sin(polar))*sph_dazi(p,q,x) - (np.sin(azimuth)/np.sin(polar))*sph_dazi2(p,q,x) \
                 + np.cos(azimuth)*np.sin(polar)*sph_dpolar(p,q,x) + (1/np.tan(polar))*sph_dazi_dpolar(p,q,x)))
    
    azimuth_ = (1/r)*(sph(p,q,x)*(sph_b(q,x)*((1/np.tan(polar))*(1/np.sin(polar)) - np.sin(azimuth)) \
                           - r*((4*np.cos(polar) + np.sin(azimuth))*sph_b_dr(q,x) + r*np.cos(polar)*sph_b_dr2(q,x))) \
               + r*np.cos(polar)*sph_b_dr(q,x)*sph_dazi(p,q,x) \
               + sph_b(q,x)*(np.cos(azimuth)*sph_dazi(p,q,x) + (np.cos(azimuth) - np.cos(polar)*(1/np.tan(polar))+ 2*np.sin(polar))*sph_dpolar(p,q,x) \
                 + np.sin(azimuth)*sph_dpolar_dazi(p,q,x) - np.cos(polar)*sph_dpolar2(p,q,x)))
    return [(1/(1j*k)) * r_*np.sin(polar_)*np.cos(azimuth_), (1/(1j*k)) * r_*np.sin(polar_)*np.sin(azimuth_), (1/(1j*k)) * r_*np.cos(polar)]


In [11]:
coeff_list_temp = np.zeros(((deg+1)**2-1,2),dtype = complex)
i = 0
for q in range(1,deg+1):
    for p in range(-q,q+1):
        coeff_list_temp[i,:] = coeff_inc(p,q)
        i += 1
coeff_list = np.hstack((coeff_list_temp[:,0],coeff_list_temp[:,1]))

In [12]:
test_pt = [1,2,3]

fun_value_list_temp1 = np.zeros(((deg+1)**2-1,3),dtype = complex)
j1 = 0
for q in range(1,deg+1):
    for p in range(-q,q+1):
        fun_value_list_temp1[j1,:] = sph_wf_M(p,q,test_pt)
        j1 += 1
        
fun_value_list_temp2 = np.zeros(((deg+1)**2-1,3),dtype = complex)
j2 = 0
for q in range(1,deg+1):
    for p in range(-q,q+1):
        fun_value_list_temp2[j2,:] = sph_wf_N(p,q,test_pt)
        j2 += 1
        
fun_value_list = np.vstack((fun_value_list_temp1,fun_value_list_temp2))

In [13]:
inc_appro = [0,0,0]
inc_appro[0] = np.dot(coeff_list,fun_value_list[:,0])
inc_appro[1] = np.dot(coeff_list,fun_value_list[:,1])
inc_appro[2] = np.dot(coeff_list,fun_value_list[:,2])

print(inc_appro)

[(0.008793942371079803+0.03227380807061249j), (0.03709588346608832+0.10406446862156894j), (-0.09316961834706092-0.4315118479872672j)]


In [13]:
inc_exact =  np.exp(1j*k*np.dot(test_pt,d0))
print(inc_exact)

(-0.7596879128588213-0.6502878401571168j)


In [ ]:
def sph_dpolar_dazi(p,q,x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = sph(p, q, x)
    y3 = sph(-p, q, x)
    
    if abs(p) == q and p >=0:
        return (1j*(p**2)/np.tan(polar)) * y1
    elif abs(p) != q and p >=0:
        y2 = sph(p + 1, q, x)
        return (1j*(p**2)/np.tan(polar)) * y1 - np.sqrt((q - p)*(q + p + q)) * 1j * p * np.exp(-1j * azimuth) * y2
    elif abs(p) == q and p < 0:
        return (-1j * (p**2) / np.tan(polar)) * y3 * np.exp(2*1j*p*azimuth)*((-1)**p)
    elif abs(p) != q and p < 0:
        y4 = sph(-p + 1, q, x)
        return ((-p / np.tan(polar)) * y3 - np.sqrt((q - (-p)) * (q + (-p) + 1)) * np.exp(-1j * azimuth) * y4) * np.exp(1j*p*azimuth*2) * ((-1)**p) * (1j*p)